In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
import torchtext.experimental
import torchtext.experimental.vectors
from torchtext.experimental.datasets.raw.text_classification import RawTextIterableDataset
from torchtext.experimental.datasets.text_classification import TextClassificationDataset
from torchtext.experimental.functional import sequential_transforms, vocab_func, totensor

import collections
import random
import time

In [2]:
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

In [4]:
raw_train_data = list(raw_train_data)
raw_test_data = list(raw_test_data)

In [5]:
raw_train_data[0]

('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
raw_test_data[0]

('neg',
 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [7]:
print(f'Number of training examples: {len(raw_train_data):,}')
print(f'Number of testing examples: {len(raw_test_data):,}')

Number of training examples: 25,000
Number of testing examples: 25,000


In [8]:
def get_train_valid_split(raw_train_data, split_ratio = 0.7):
        
    random.shuffle(raw_train_data)
        
    n_train_examples = int(len(raw_train_data) * split_ratio)
        
    train_data = raw_train_data[:n_train_examples]
    valid_data = raw_train_data[n_train_examples:]
    
    return train_data, valid_data

In [9]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [10]:
print(f'Number of training examples: {len(raw_train_data):,}')
print(f'Number of validation examples: {len(raw_valid_data):,}')
print(f'Number of testing examples: {len(raw_test_data):,}')

Number of training examples: 17,500
Number of validation examples: 7,500
Number of testing examples: 25,000


In [11]:
class Tokenizer:
    def __init__(self, tokenize_fn = 'basic_english', lower = True, max_length = None):
        
        self.tokenize_fn = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, s):
        
        tokens = self.tokenize_fn(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.max_length is not None:
            tokens = tokens[:self.max_length]
            
        return tokens

In [12]:
max_length = 250

tokenizer = Tokenizer(max_length = max_length)

In [13]:
s = "this film is terrible. i hate it and it's bad!"

print(tokenizer.tokenize(s))

['this', 'film', 'is', 'terrible', '.', 'i', 'hate', 'it', 'and', 'it', "'", 's', 'bad', '!']


In [14]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
        
    token_freqs = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freqs.update(tokens)
                
    vocab = torchtext.vocab.Vocab(token_freqs, **vocab_kwargs)
    
    return vocab

In [15]:
max_size = 25_000

vocab = build_vocab_from_data(raw_train_data, tokenizer, max_size = max_size)

In [16]:
print(f'Unique tokens in vocab: {len(vocab):,}')

Unique tokens in vocab: 25,002


In [17]:
vocab.freqs.most_common(20)

[('the', 165322),
 ('.', 164239),
 (',', 133647),
 ('a', 81952),
 ('and', 80334),
 ('of', 71820),
 ('to', 65662),
 ("'", 64249),
 ('is', 53598),
 ('it', 49589),
 ('i', 48810),
 ('in', 45611),
 ('this', 40868),
 ('that', 35609),
 ('s', 29273),
 ('was', 26159),
 ('movie', 24543),
 ('as', 22276),
 ('with', 21494),
 ('for', 21332)]

In [18]:
vocab.itos[:10]

['<unk>', '<pad>', 'the', '.', ',', 'a', 'and', 'of', 'to', "'"]

In [19]:
vocab.stoi['the']

2

In [20]:
def raw_data_to_dataset(raw_data, tokenizer, vocab):
        
    text_transform = sequential_transforms(tokenizer.tokenize,
                                           vocab_func(vocab),
                                           totensor(dtype=torch.long))
    
    label_transform = sequential_transforms(lambda x: 1 if x == 'pos' else 0, 
                                            totensor(dtype=torch.long))

    transforms = (label_transform, text_transform)

    dataset = TextClassificationDataset(raw_data,
                                        vocab,
                                        transforms)
    
    return dataset

In [21]:
train_data = raw_data_to_dataset(raw_train_data, tokenizer, vocab)
valid_data = raw_data_to_dataset(raw_valid_data, tokenizer, vocab)
test_data = raw_data_to_dataset(raw_test_data, tokenizer, vocab)

In [22]:
print(f'Number of training examples: {len(train_data):,}')
print(f'Number of validation examples: {len(valid_data):,}')
print(f'Number of testing examples: {len(test_data):,}')

Number of training examples: 17,500
Number of validation examples: 7,500
Number of testing examples: 25,000


In [23]:
label, indexes = test_data[0]

print(indexes)

tensor([   12,   121,  1013,     6,   219,  1855,     8,   276,    70,    20,
            5,   177,     3,  1013,     0,    30,   541,     0,     4, 15259,
            6,  7022,     3,    12,   751,     8,    45,    14,     4,    12,
           69,   123,     4,    22,    11,    10,     8,    56,   241,  1013,
           19, 12534,   563,    10,     8,   338,  1803,    25,     2,   196,
           24,     3,   717,     0,     4,   745,  3428,   686,     4,  4315,
         3437,     4,  4258,    15,   170,     9,    28,  1209,     2,   951,
            4,     6,  2005,  5083,   113,   544,    35,  2957,    20,     5,
            9,  1013,     9,   925,     3,    25,    12,     9,   145,   255,
           46,    30,   160,     7,    26,    54,    46,    42,   107, 12534,
          563,    10,    56,  1013,   241,     3,    11,     9,    16,    29,
            3,    11,     9,    16,  2966,     6,  8018,     3,    24,   143,
          199,   773,   249,    45,  1364,     6,   120,   893, 

In [24]:
print([vocab.itos[i] for i in indexes])

['i', 'love', 'sci-fi', 'and', 'am', 'willing', 'to', 'put', 'up', 'with', 'a', 'lot', '.', 'sci-fi', '<unk>', 'are', 'usually', '<unk>', ',', 'under-appreciated', 'and', 'misunderstood', '.', 'i', 'tried', 'to', 'like', 'this', ',', 'i', 'really', 'did', ',', 'but', 'it', 'is', 'to', 'good', 'tv', 'sci-fi', 'as', 'babylon', '5', 'is', 'to', 'star', 'trek', '(', 'the', 'original', ')', '.', 'silly', '<unk>', ',', 'cheap', 'cardboard', 'sets', ',', 'stilted', 'dialogues', ',', 'cg', 'that', 'doesn', "'", 't', 'match', 'the', 'background', ',', 'and', 'painfully', 'one-dimensional', 'characters', 'cannot', 'be', 'overcome', 'with', 'a', "'", 'sci-fi', "'", 'setting', '.', '(', 'i', "'", 'm', 'sure', 'there', 'are', 'those', 'of', 'you', 'out', 'there', 'who', 'think', 'babylon', '5', 'is', 'good', 'sci-fi', 'tv', '.', 'it', "'", 's', 'not', '.', 'it', "'", 's', 'clichéd', 'and', 'uninspiring', '.', ')', 'while', 'us', 'viewers', 'might', 'like', 'emotion', 'and', 'character', 'developmen

In [25]:
class Collator:
    def __init__(self, pad_idx):
        
        self.pad_idx = pad_idx
        
    def collate(self, batch):
        
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)
        
        text = nn.utils.rnn.pad_sequence(text, padding_value = self.pad_idx)
        
        return labels, text

In [26]:
pad_token = '<pad>'
pad_idx = vocab[pad_token]

collator = Collator(pad_idx)

In [27]:
batch_size = 256

train_iterator = torch.utils.data.DataLoader(train_data, 
                                             batch_size, 
                                             shuffle = True, 
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size, 
                                             shuffle = False, 
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size, 
                                            shuffle = False, 
                                            collate_fn = collator.collate)

In [28]:
class NBOW(nn.Module):
    def __init__(self, input_dim, emb_dim, output_dim, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.fc = nn.Linear(emb_dim, output_dim)
        
    def forward(self, text):
        
        # text = [seq len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [seq len, batch size, emb dim]
        
        pooled = embedded.mean(0)
        
        # pooled = [batch size, emb dim]
        
        prediction = self.fc(pooled)
        
        # prediction = [batch size, output dim]
        
        return prediction

In [29]:
input_dim = len(vocab)
emb_dim = 100
output_dim = 2

model = NBOW(input_dim, emb_dim, output_dim, pad_idx)

In [30]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [31]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,402 trainable parameters


In [32]:
glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)

In [33]:
glove['the']

tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  0.8278,  0.27

In [34]:
glove['shoggoth']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [35]:
glove['The']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [36]:
glove_vocab = glove.vectors.get_stoi()

In [37]:
'the' in glove_vocab

True

In [38]:
'The' in glove_vocab

False

In [39]:
def get_pretrained_embedding(initial_embedding, pretrained_vectors, vocab, unk_token):
    
    pretrained_embedding = torch.FloatTensor(initial_embedding.weight.clone()).detach()    
    pretrained_vocab = pretrained_vectors.vectors.get_stoi()
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        if token in pretrained_vocab:
            pretrained_vector = pretrained_vectors[token]
            pretrained_embedding[idx] = pretrained_vector
        else:
            unk_tokens.append(token)
        
    return pretrained_embedding, unk_tokens

In [40]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(model.embedding, glove, vocab, unk_token)

In [41]:
model.embedding.weight.data

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7289, -0.7336,  1.5624,  ..., -0.5592, -0.4480, -0.6476],
        ...,
        [ 0.0914,  1.5196,  0.4670,  ...,  0.6393, -0.0332,  0.0185],
        [-0.6290,  0.4650, -0.7165,  ..., -1.3171,  2.0381, -2.0497],
        [-1.1222, -0.0240, -1.0878,  ..., -0.4948, -0.3874,  0.0339]])

In [42]:
pretrained_embedding

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4029,  0.1353,  0.6673,  ..., -0.3300,  0.7533, -0.1666],
        [ 0.1226,  0.0419,  0.0746,  ..., -0.0024, -0.2733, -1.0033],
        [-0.1009, -0.1484,  0.3141,  ..., -0.3414, -0.3768,  0.5605]])

In [43]:
len(unk_tokens)

734

In [44]:
print(unk_tokens[:10])

['<unk>', '<pad>', '\x96', '****', 'hadn', 'camera-work', '*1/2', '100%', '*****', '$1']


In [45]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.5903, -0.1947, -0.2415],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4029,  0.1353,  0.6673,  ..., -0.3300,  0.7533, -0.1666],
        [ 0.1226,  0.0419,  0.0746,  ..., -0.0024, -0.2733, -1.0033],
        [-0.1009, -0.1484,  0.3141,  ..., -0.3414, -0.3768,  0.5605]])

In [46]:
optimizer = optim.Adam(model.parameters())

In [47]:
criterion = nn.CrossEntropyLoss()

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using: {device}')

Using: cuda


In [49]:
model = model.to(device)
criterion = criterion.to(device)

In [50]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [51]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text in iterator:
        
        labels = labels.to(device)
        text = text.to(device)
        
        optimizer.zero_grad()
        
        predictions = model(text)
        
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [53]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [54]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'nbow-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 0.683 | Train Acc: 60.00%
	 Val. Loss: 0.669 |  Val. Acc: 67.02%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss: 0.651 | Train Acc: 68.09%
	 Val. Loss: 0.632 |  Val. Acc: 71.31%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss: 0.603 | Train Acc: 74.06%
	 Val. Loss: 0.582 |  Val. Acc: 74.86%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss: 0.545 | Train Acc: 78.13%
	 Val. Loss: 0.528 |  Val. Acc: 78.88%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss: 0.485 | Train Acc: 82.10%
	 Val. Loss: 0.477 |  Val. Acc: 81.64%
Epoch: 06 | Epoch Time: 0m 4s
	Train Loss: 0.430 | Train Acc: 85.15%
	 Val. Loss: 0.437 |  Val. Acc: 83.25%
Epoch: 07 | Epoch Time: 0m 4s
	Train Loss: 0.386 | Train Acc: 86.92%
	 Val. Loss: 0.404 |  Val. Acc: 84.59%
Epoch: 08 | Epoch Time: 0m 4s
	Train Loss: 0.350 | Train Acc: 88.21%
	 Val. Loss: 0.383 |  Val. Acc: 85.19%
Epoch: 09 | Epoch Time: 0m 4s
	Train Loss: 0.319 | Train Acc: 89.36%
	 Val. Loss: 0.363 |  Val. Acc: 85.86%
Epoch: 10 | Epoch Time: 0m 4

In [55]:
model.load_state_dict(torch.load('nbow-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.374 | Test Acc: 84.75%


In [56]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [57]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

2.818893153744284e-05

In [58]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9997795224189758

In [59]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.6041761040687561

In [60]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.6041760444641113